# ЖКХ- бот 
для жилищно-коммунального хозяйства
для управляющих компаний и ТСЖ

In [1]:
!pip install pytelegrambotapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 217 kB 5.1 MB/s 
  Created wheel for pytelegrambotapi: filename=pyTelegramBotAPI-4.8.0-py3-none-any.whl size=200125 sha256=aebe06080ce538bd319ee2efa744eeb2e3017f31d22c9bb268c88acba3d31ca3
  Stored in directory: /root/.cache/pip/wheels/c2/05/70/8409792e663e67a70e057df1f18d070105c1c457b3f410bbb0
Successfully built pytelegrambotapi


In [26]:
import pandas as pd
import telebot, re
from telebot import types
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

In [27]:
col = ['лиц.счет','елс','улица','дом','кв','месяц','сумма']
df = pd.read_csv('/content/0571005074.txt', dtype=str, encoding='cp1251', sep=';|,', header=None, engine='python', usecols=[0,1,6,7,8,9,10], names=col )
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x) # удаляем пробелы до и после значений во всём датафрейме

In [12]:
df.head()

,лиц.счет,елс,улица,дом,кв,месяц,сумма
0,01401,20ОХ277032,Бейбулатова,14,1,1122,2151.5
1,01402,10ОХ277201,Бейбулатова,14,2,1122,4851.61
2,01403,00ОХ277218,Бейбулатова,14,3,1122,4851.61
3,01404,20ОХ277155,Бейбулатова,14,4,1122,3589.9
4,01405,50ОХ277158,Бейбулатова,14,5,1122,-715


In [ ]:
# объявляем переменные
bot = telebot.TeleBot('свой ключ для бота')
kv = ''
dm = ''
ul = ''
question = ""
b = 'Бейбулатова'
p = 'Петра 1'
sh = 'Шамиля'
g = 'Гамидова'

def sent(lc): # функция нахождения данных  по лицевому счету
    i =df[df['лиц.счет']== lc].index[0]
    ul = df['улица'].values[i]
    d = df['дом'].values[i]
    kv = df['кв'].values[i]
    m = df['месяц'].values[i]
    s = df['сумма'].values[i]
    return ul,d,kv,m,s

def out(g):  # вывод информации долг, нет долгов или переплата в зависимости от суммы.
    if float(g[4])>0:
        x = (' задолженность '+g[4]+' рублей включая(месяц,год)'+g[3])
    elif float(g[4])<0:
        x = (' переплата'+g[4]+' рублей включая(месяц,год)'+g[3])
    else:
        x = (' нет долгов включая(месяц,год)'+g[3])
    return ('по адресу: улица '+g[0]+', д.'+g[1]+', квартира '+g[2]+x)

def clean_str(r): # к нижнему регистру и убрать лишнее
    r = r.lower()
    r = [c for c in r if c in alphabet]
    return ''.join(r)

alphabet = ' 1234567890-йцукенгшщзхъфывапролджэячсмитьбюёqwertyuiopasdfghjklzxcvbnm?%.,()!:;'

def update(): # Функция подготовки векторов из файла данных 
    with open('dialogues.txt', encoding='utf-8') as f: # Читаем файл с заготовками
        content = f.read() # Читаем файл
	
    blocks = content.split('\n') # создаем список вопросы\ответы
    dataset = [] # создаем пустой список
	
    for block in blocks:  # перебираем список 
        replicas = block.split('\\')[:2] # заносим вопрос-ответ в переменную
        if len(replicas) == 2: # проверка список из двух?
            pair = [clean_str(replicas[0]), clean_str(replicas[1])] # Убираем лишнее
            if pair[0] and pair[1]:  
                dataset.append(pair) # Создаем список списков вопросы-ответы
	
    X_text = []  
    y = []
	
    for question, answer in dataset[:10000]: # перебираем датасет
        X_text.append(question) # список вопросов
        y += [answer] # список ответов
	
    global vectorizer # для сохранения переменной вне функции def update()
    vectorizer = CountVectorizer() # Метод вносим в переменую
    X = vectorizer.fit_transform(X_text) # преобразования заданного текста в вектор на основе частоты (количества) каждого слова
	
    global clf
    clf = LogisticRegression() # Метод вносим в переменую
    clf.fit(X, y) # Логистическая регрессия
    	

update()


@bot.message_handler(commands=['start'])  # стартовый блок от Телеграмма
def start_message(message):
      
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True) #создание новых кнопок
    btn1 = types.KeyboardButton('Справка')
    btn2 = types.KeyboardButton('Долг')
    btn3 = types.KeyboardButton('Оплата')
    markup.add(btn1, btn2, btn3)
    pr =  'Здравствуйте! \nУ меня есть три кнопки на выбор.'   
    bot.send_message(message.from_user.id, pr, reply_markup=markup) #ответ бота    

def get_generative_replica(text): # Функция нахождения похожего вектора слова
    text_vector = vectorizer.transform([text]).toarray()[0]
    question = clf.predict([text_vector])[0]
    return question

@bot.message_handler(content_types=['text']) # текстовый блок от Телеграмма
def start_lc(message):    
    if message.text == 'Справка':
        bot.send_message(message.from_user.id, 'Спрашивайте?')
        bot.register_next_step_handler(message, get_text_messages)
        
    elif message.text == 'Долг':
        bot.send_message(message.from_user.id, 'Введите 5 значный лицевой счёт')
        bot.register_next_step_handler(message, ask_ge)

    elif message.text == 'Оплата': 
        txt = '''оплатить можно через приложение сбербанка — 
        раздел платежи — дом — квартплата, 
        далее вверху набрать Белгравия и поиск — 
        потом набрать ваш лицевой счёт'''
        bot.send_message(message.from_user.id, txt)


def get_text_messages(message): # Функция ответов на вопросы и пополнения базы данных 
    command = message.text.lower()
    if  command =="не так":
        bot.send_message(message.from_user.id, "а как?")
        bot.register_next_step_handler(message, wrong)
    
    elif message.text in ['Справка','Долг', 'Оплата']:
         start_lc(message)
        
    else:
        global question
        question = command
        reply = get_generative_replica(command)
        bot.send_message(message.from_user.id, reply) 
        bot.register_next_step_handler(message, get_text_messages)

def wrong(message): #  функция внесения поправок в файл
    a = f"{question}\{message.text.lower()} \n"
    with open('dialogues.txt', "a", encoding='utf-8') as f:
        f.write(a)
    bot.send_message(message.from_user.id, "Ок, записал")
    update()
    bot.register_next_step_handler(message, get_text_messages)

def ask_ge(message): # выбор и проверка лицевого счета
    if  message.text in ['Справка','Долг', 'Оплата']:
        start_lc(message)     
    elif not message.text.isdigit(): # проверка на число 
        bot.send_message(message.chat.id, 'Лицевой счёт должен быть числом, введите ещё раз.')
        bot.register_next_step_handler(message, ask_ge)
    elif message.text in df['лиц.счет'].unique():
        bot.send_message(message.chat.id, out(sent(message.text)))
        bot.register_next_step_handler(message, ask_ge)
    else:
        markup = types.InlineKeyboardMarkup() #клавиатура помощи 
        key_yes = types.InlineKeyboardButton(text='Да', callback_data='yes') #кнопка «Да»
        key_no= types.InlineKeyboardButton(text='Нет', callback_data='no')
        markup.add(key_yes, key_no) #добавляем кнопки в клавиатуру
        question = 'В базе данных нет такого счёта, Вам помочь?'
        bot.send_message(message.chat.id, text=question, reply_markup=markup)

def kvar(message): 
    global kv
    kv = message.text
    if kv in df['кв'].values: # проверка существования квартиры в файле
        bot.send_message(message.from_user.id, "Наберите номер дома")
        bot.register_next_step_handler(message, dom)
    else:
        bot.send_message(message.from_user.id, "Нет такой квартиры, напишите еще раз")
        bot.register_next_step_handler(message, kvar)

def dom(message): 
    global dm
    dm = message.text.lower() # к нижнему регистру и сообщение и датафрейм
    if dm in df['дом'].str.lower().values:# проверка существования номера дома
        keyboard = types.InlineKeyboardMarkup() # запускаем кнопки выбора улицы
        key1 = types.InlineKeyboardButton(b, callback_data=b)
        keyboard.add(key1)
        key2 = types.InlineKeyboardButton(sh, callback_data=sh)
        keyboard.add(key2)
        key3 = types.InlineKeyboardButton(p, callback_data=p)
        keyboard.add(key3)
        key4 = types.InlineKeyboardButton(g, callback_data=g)
        keyboard.add(key4)
        bot.send_message(message.from_user.id, 'Нажмите кнопку с улицей', reply_markup=keyboard)
       
    else:
        bot.send_message(message.from_user.id, "Нет такого дома в нашей базе, напишите еще раз")
        bot.register_next_step_handler(message, dom)

def reg_ad(message): # Провека существования лицевого счёта кв+дом+улица 
    rg = df.loc[df.isin([ul]).any(axis=1) & 
                df.isin([kv]).any(axis=1) & 
                df.isin([dm]).any(axis=1)].empty
    if  rg != True:
        in_lch = df.loc[(df['кв'] == kv) & 
                        (df['дом'].str.lower().values == dm) & 
                        (df['улица'] == ul)].index[0]
        lch = df['лиц.счет'].loc[df.index[in_lch]]
        sen_t = 'ваш лицевой счёт '+ lch + '\n'  + out(sent(lch))
        bot.send_message(message.from_user.id, sen_t)
    else:
        txt = "Нет такого варианта в базе, начните снова с выбора кнопок"
        bot.send_message(message.from_user.id, txt)

@bot.callback_query_handler(func=lambda call: True) 
def callback_worker(call):# функция callback для кнопок 
  global ul
  try:  
    if call.message: 
        if call.data == "yes": #call.data это callback_data, которую мы указали при объявлении кнопки
            txt = 'Напишите номер вашей квартиры?'
            bot.edit_message_text(chat_id=call.message.chat.id, message_id=call.message.id, text=txt)
            bot.answer_callback_query(call.id)
            bot.register_next_step_handler(call.message, kvar)

        if call.data == "no":            
            bot.edit_message_text(chat_id=call.message.chat.id, message_id=call.message.id, text="Пробуем снова") 
            bot.answer_callback_query(call.id)
            bot.register_next_step_handler(call.message, ask_ge)

        if call.data == b:
            ul = b
        if call.data == p:
            ul = p            
        if call.data == sh:
            ul = sh
        if call.data == g:
            ul = g
        bot.edit_message_text(chat_id=call.message.chat.id, message_id=call.message.message_id, text=ul)
        bot.answer_callback_query(call.id)
        reg_ad(call)        
  except Exception as e:
	  print(repr(e))

bot.polling(none_stop=True) #цикл для постоянной работы бота